In [3]:
import pandas as pd
import pyodbc
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Database connection parameters
DATA_SOURCE = "100.200.2.1"
DATABASE_PATH = r"D:\dolly2008\fer2015.dol"
USERNAME = "ALIOSS"
PASSWORD = "$9-j[+Mo$AA833C4FA$"
CLIENT_LIBRARY = r"C:\Users\User\Downloads\Compressed\ibclient64-14.1_x86-64\ibclient64-14.1.dll"

connection_string = (
    f"DRIVER=Devart ODBC Driver for InterBase;"
    f"Data Source={DATA_SOURCE};"
    f"Database={DATABASE_PATH};"
    f"User ID={USERNAME};"
    f"Password={PASSWORD};"
    f"Client Library={CLIENT_LIBRARY};"
)

def connect_and_load_table(table_name):
    """Load a table from the database using manual DataFrame creation"""
    try:
        print(f"🔄 Connecting to database for table {table_name}...")
        conn = pyodbc.connect(connection_string)
        cursor = conn.cursor()
        print(f"✅ Connected successfully, loading {table_name}...")
        
        # Execute query and get column names
        cursor.execute(f"SELECT * FROM {table_name}")
        columns = [column[0] for column in cursor.description]
        rows = cursor.fetchall()
        
        # Convert to DataFrame manually
        df = pd.DataFrame([list(row) for row in rows], columns=columns)
        
        conn.close()
        print(f"✅ {table_name}: {df.shape[0]:,} rows × {df.shape[1]} columns")
        return df
    except Exception as e:
        print(f"❌ {table_name}: Failed to load - {e}")
        print(f"   Connection string: {connection_string[:50]}...")
        return None

print("✅ Connection setup complete")

✅ Connection setup complete


In [4]:
# Load tables with descriptive names
print("Loading database tables...")

sites_df = connect_and_load_table('ALLSTOCK')          # Site/Location master data
categories_df = connect_and_load_table('DETDESCR')     # Category definitions  
invoice_headers_df = connect_and_load_table('INVOICE') # Invoice headers
sales_details_df = connect_and_load_table('ITEMS')     # Sales transaction details
vouchers_df = connect_and_load_table('PAYM')           # Payment vouchers
accounts_df = connect_and_load_table('SACCOUNT')       # Statement of accounts
inventory_items_df = connect_and_load_table('STOCK')   # Items/Products master
inventory_transactions_df = connect_and_load_table('ALLITEM') # All inventory transactions

# Create dataframes dictionary with descriptive names
dataframes = {
    'sites': sites_df,
    'categories': categories_df, 
    'invoice_headers': invoice_headers_df,
    'sales_details': sales_details_df,
    'vouchers': vouchers_df,
    'accounts': accounts_df,
    'inventory_items': inventory_items_df,
    'inventory_transactions': inventory_transactions_df
}

# Remove None values and show summary
dataframes = {k: v for k, v in dataframes.items() if v is not None}
print(f"\n✅ Successfully loaded {len(dataframes)} tables:")
for name, df in dataframes.items():
    print(f"  {name}: {df.shape[0]:,} rows × {df.shape[1]} columns")

Loading database tables...
🔄 Connecting to database for table ALLSTOCK...
✅ Connected successfully, loading ALLSTOCK...
✅ ALLSTOCK: 155 rows × 18 columns
🔄 Connecting to database for table DETDESCR...
✅ Connected successfully, loading DETDESCR...
✅ DETDESCR: 159 rows × 10 columns
🔄 Connecting to database for table INVOICE...
✅ Connected successfully, loading INVOICE...
✅ INVOICE: 207,949 rows × 54 columns
🔄 Connecting to database for table ITEMS...
✅ Connected successfully, loading ITEMS...
✅ INVOICE: 207,949 rows × 54 columns
🔄 Connecting to database for table ITEMS...
✅ Connected successfully, loading ITEMS...
✅ ITEMS: 2,405,320 rows × 54 columns
🔄 Connecting to database for table PAYM...
✅ Connected successfully, loading PAYM...
✅ ITEMS: 2,405,320 rows × 54 columns
🔄 Connecting to database for table PAYM...
✅ Connected successfully, loading PAYM...
✅ PAYM: 192,430 rows × 41 columns
🔄 Connecting to database for table SACCOUNT...
✅ Connected successfully, loading SACCOUNT...
✅ PAYM: 1

In [22]:
# Test connection first
print("Testing database connection...")
try:
    test_conn = pyodbc.connect(connection_string)
    test_conn.close()
    print("✅ Database connection test successful")
except Exception as e:
    print(f"❌ Database connection test failed: {e}")
    print(f"Connection string: {connection_string}")

# Test loading just one table
print("\nTesting one table load...")
test_df = connect_and_load_table('ALLSTOCK')

Testing database connection...
✅ Database connection test successful

Testing one table load...
🔄 Connecting to database for table ALLSTOCK...
✅ Connected successfully, loading ALLSTOCK...
✅ ALLSTOCK: 155 rows × 18 columns


In [23]:
# Test direct pyodbc approach
print("Testing direct pyodbc query...")
try:
    conn = pyodbc.connect(connection_string)
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM ALLSTOCK")
    rows = cursor.fetchmany(5)  # Just get first 5 rows
    print(f"✅ Direct query successful - got {len(rows)} rows")
    print(f"Columns: {[column[0] for column in cursor.description]}")
    conn.close()
except Exception as e:
    print(f"❌ Direct query failed: {e}")

# Test manual DataFrame creation
print("\nTesting manual DataFrame creation...")
try:
    conn = pyodbc.connect(connection_string)
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM ALLSTOCK")
    columns = [column[0] for column in cursor.description]
    rows = cursor.fetchall()
    
    # Convert to DataFrame manually
    df = pd.DataFrame([list(row) for row in rows], columns=columns)
    print(f"✅ Manual DataFrame creation successful: {df.shape[0]:,} rows × {df.shape[1]} columns")
    print(f"Columns: {list(df.columns)}")
    conn.close()
except Exception as e:
    print(f"❌ Manual DataFrame creation failed: {e}")

Testing direct pyodbc query...
✅ Direct query successful - got 5 rows
Columns: ['ID', 'SITE', 'STACTIVE', 'SIDNO', 'SID606', 'MYCHECK', 'TEL', 'MYCHECK2', 'PLACE', 'FREIGHT', 'TARGET', 'PERCENTAGE', 'PERCENTAGE2', 'SID2', 'MYORDER', 'MYNAME', 'PERC', 'JOB']

Testing manual DataFrame creation...
✅ Manual DataFrame creation successful: 155 rows × 18 columns
Columns: ['ID', 'SITE', 'STACTIVE', 'SIDNO', 'SID606', 'MYCHECK', 'TEL', 'MYCHECK2', 'PLACE', 'FREIGHT', 'TARGET', 'PERCENTAGE', 'PERCENTAGE2', 'SID2', 'MYORDER', 'MYNAME', 'PERC', 'JOB']


In [7]:
# Stock Calculation Functions

def calculate_stock_and_sales(item_code=None, site_code=None, from_date=None, to_date=None, show_details=False):
    """
    Calculate current stock and sales for an item at a specific site or across all sites
    
    Parameters:
    - item_code: Item code (if None, calculates for all items)
    - site_code: Site code (if None, calculates across all sites)
    - from_date: Start date for sales calculation (format: 'YYYY-MM-DD')
    - to_date: End date for sales calculation (format: 'YYYY-MM-DD')
    - show_details: If True, shows detailed breakdown
    
    Returns: DataFrame with stock and sales calculations
    """
    from datetime import datetime
    
    # Start with inventory transactions for stock calculation
    df_stock = inventory_transactions_df.copy()
    
    # Filter by item if specified
    if item_code:
        df_stock = df_stock[df_stock['ITEM'] == item_code]
        if df_stock.empty:
            print(f"❌ No stock transactions found for item: {item_code}")
            return None
    
    # Filter by site if specified  
    if site_code:
        df_stock = df_stock[df_stock['SITE'] == site_code]
        if df_stock.empty:
            print(f"❌ No stock transactions found for site: {site_code}")
            return None
    
    # Fill NaN values with 0 for calculations
    df_stock['DEBITQTY'] = df_stock['DEBITQTY'].fillna(0)
    df_stock['CREDITQTY'] = df_stock['CREDITQTY'].fillna(0)
    
    # Calculate stock by grouping
    if item_code and site_code:
        # Single item, single site
        result_df = pd.DataFrame({
            'SITE': [site_code],
            'ITEM': [item_code],
            'TOTAL_IN': [df_stock['DEBITQTY'].sum()],
            'TOTAL_OUT': [df_stock['CREDITQTY'].sum()],
            'CURRENT_STOCK': [df_stock['DEBITQTY'].sum() - df_stock['CREDITQTY'].sum()],
            'STOCK_TRANSACTIONS': [len(df_stock)]
        })
    elif item_code:
        # Single item, all sites
        result_df = df_stock.groupby('SITE').agg({
            'DEBITQTY': 'sum',
            'CREDITQTY': 'sum',
            'ITEM': 'first'
        }).reset_index()
        result_df['CURRENT_STOCK'] = result_df['DEBITQTY'] - result_df['CREDITQTY']
        result_df['STOCK_TRANSACTIONS'] = df_stock.groupby('SITE').size().values
        result_df = result_df.rename(columns={'DEBITQTY': 'TOTAL_IN', 'CREDITQTY': 'TOTAL_OUT'})
        result_df = result_df[['SITE', 'ITEM', 'TOTAL_IN', 'TOTAL_OUT', 'CURRENT_STOCK', 'STOCK_TRANSACTIONS']]
    elif site_code:
        # All items, single site
        result_df = df_stock.groupby('ITEM').agg({
            'DEBITQTY': 'sum',
            'CREDITQTY': 'sum',
            'SITE': 'first'
        }).reset_index()
        result_df['CURRENT_STOCK'] = result_df['DEBITQTY'] - result_df['CREDITQTY']
        result_df['STOCK_TRANSACTIONS'] = df_stock.groupby('ITEM').size().values
        result_df = result_df.rename(columns={'DEBITQTY': 'TOTAL_IN', 'CREDITQTY': 'TOTAL_OUT'})
        result_df = result_df[['SITE', 'ITEM', 'TOTAL_IN', 'TOTAL_OUT', 'CURRENT_STOCK', 'STOCK_TRANSACTIONS']]
    else:
        # All items, all sites
        result_df = df_stock.groupby(['SITE', 'ITEM']).agg({
            'DEBITQTY': 'sum',
            'CREDITQTY': 'sum'
        }).reset_index()
        result_df['CURRENT_STOCK'] = result_df['DEBITQTY'] - result_df['CREDITQTY']
        result_df['STOCK_TRANSACTIONS'] = df_stock.groupby(['SITE', 'ITEM']).size().values
        result_df = result_df.rename(columns={'DEBITQTY': 'TOTAL_IN', 'CREDITQTY': 'TOTAL_OUT'})
    
    # Calculate sales analytics from sales_details_df (ITEMS table)
    if 'sales_details' in dataframes and sales_details_df is not None:
        df_sales = sales_details_df.copy()
        
        # Filter sales by item and site
        if item_code:
            df_sales = df_sales[df_sales['ITEM'] == item_code]
        if site_code:
            df_sales = df_sales[df_sales['SITE'] == site_code]
        
        # Convert FDATE to datetime if it's not already
        if 'FDATE' in df_sales.columns:
            df_sales['FDATE'] = pd.to_datetime(df_sales['FDATE'], errors='coerce')
            
            # Filter by date range if specified
            if from_date:
                from_date_dt = pd.to_datetime(from_date)
                df_sales = df_sales[df_sales['FDATE'] >= from_date_dt]
            if to_date:
                to_date_dt = pd.to_datetime(to_date)
                df_sales = df_sales[df_sales['FDATE'] <= to_date_dt]
        
        # Filter for sales transactions (FTYPE = 1 for sales, FTYPE = 2 for returns)
        if 'FTYPE' in df_sales.columns:
            sales_only = df_sales[df_sales['FTYPE'].isin([1, 2])]
        else:
            sales_only = df_sales
        
        # Fill NaN values
        sales_only['QTY'] = sales_only['QTY'].fillna(0)
        
        # Calculate daily sales analytics
        def calculate_sales_analytics(group, from_date_param=None, to_date_param=None):
            if group.empty or 'FDATE' not in group.columns:
                return pd.Series({
                    'MAX_DAILY_SALES': 0,
                    'MIN_DAILY_SALES': 0, 
                    'AVG_DAILY_SALES': 0,
                    'SALES_TRANSACTIONS': 0,
                    'TOTAL_SALES_QTY': 0,
                    'SALES_PERIOD_DAYS': 0
                })
            
            # Group by date and sum quantities
            daily_sales = group.groupby('FDATE')['QTY'].sum()
            
            # Calculate period metrics
            total_sales_qty = group['QTY'].sum()
            
            # Use specified date range if provided, otherwise use actual sales date range
            if from_date_param and to_date_param:
                from_dt = pd.to_datetime(from_date_param)
                to_dt = pd.to_datetime(to_date_param)
                period_days = (to_dt - from_dt).days + 1
            elif from_date_param:
                from_dt = pd.to_datetime(from_date_param)
                max_date = group['FDATE'].max()
                period_days = (max_date - from_dt).days + 1
            elif to_date_param:
                min_date = group['FDATE'].min()
                to_dt = pd.to_datetime(to_date_param)
                period_days = (to_dt - min_date).days + 1
            else:
                # No date range specified, use actual sales period
                min_date = group['FDATE'].min()
                max_date = group['FDATE'].max()
                period_days = (max_date - min_date).days + 1 if min_date != max_date else 1
            
            # Exclude zero sales days for min calculation
            non_zero_sales = daily_sales[daily_sales > 0]
            
            max_sales = daily_sales.max() if not daily_sales.empty else 0
            min_sales = non_zero_sales.min() if not non_zero_sales.empty else 0
            
            # Calculate average daily sales using the full specified period
            avg_sales = total_sales_qty / period_days if period_days > 0 else 0
            
            total_transactions = len(group)
            
            return pd.Series({
                'MAX_DAILY_SALES': max_sales,
                'MIN_DAILY_SALES': min_sales,
                'AVG_DAILY_SALES': avg_sales,
                'SALES_TRANSACTIONS': total_transactions,
                'TOTAL_SALES_QTY': total_sales_qty,
                'SALES_PERIOD_DAYS': period_days
            })
        
        # Calculate analytics by same grouping as stock
        if item_code and site_code:
            # Single item, single site
            analytics = calculate_sales_analytics(sales_only, from_date, to_date)
            sales_analytics = pd.DataFrame({
                'SITE': [site_code],
                'ITEM': [item_code],
                'MAX_DAILY_SALES': [analytics['MAX_DAILY_SALES']],
                'MIN_DAILY_SALES': [analytics['MIN_DAILY_SALES']], 
                'AVG_DAILY_SALES': [analytics['AVG_DAILY_SALES']],
                'SALES_TRANSACTIONS': [analytics['SALES_TRANSACTIONS']],
                'TOTAL_SALES_QTY': [analytics['TOTAL_SALES_QTY']],
                'SALES_PERIOD_DAYS': [analytics['SALES_PERIOD_DAYS']]
            })
        elif item_code:
            # Single item, all sites
            sales_analytics = sales_only.groupby('SITE').apply(lambda x: calculate_sales_analytics(x, from_date, to_date)).reset_index()
            sales_analytics['ITEM'] = item_code
        elif site_code:
            # All items, single site
            sales_analytics = sales_only.groupby('ITEM').apply(lambda x: calculate_sales_analytics(x, from_date, to_date)).reset_index()
            sales_analytics['SITE'] = site_code
        else:
            # All items, all sites
            sales_analytics = sales_only.groupby(['SITE', 'ITEM']).apply(lambda x: calculate_sales_analytics(x, from_date, to_date)).reset_index()
        
        # Merge stock and sales analytics
        result_df = result_df.merge(sales_analytics[['SITE', 'ITEM', 'MAX_DAILY_SALES', 'MIN_DAILY_SALES', 'AVG_DAILY_SALES', 'SALES_TRANSACTIONS', 'TOTAL_SALES_QTY', 'SALES_PERIOD_DAYS']], 
                                   on=['SITE', 'ITEM'], how='left')
        result_df['MAX_DAILY_SALES'] = result_df['MAX_DAILY_SALES'].fillna(0)
        result_df['MIN_DAILY_SALES'] = result_df['MIN_DAILY_SALES'].fillna(0)
        result_df['AVG_DAILY_SALES'] = result_df['AVG_DAILY_SALES'].fillna(0)
        result_df['SALES_TRANSACTIONS'] = result_df['SALES_TRANSACTIONS'].fillna(0)
        result_df['TOTAL_SALES_QTY'] = result_df['TOTAL_SALES_QTY'].fillna(0)
        result_df['SALES_PERIOD_DAYS'] = result_df['SALES_PERIOD_DAYS'].fillna(0)
    else:
        # Add empty sales columns if sales data not available
        result_df['MAX_DAILY_SALES'] = 0
        result_df['MIN_DAILY_SALES'] = 0
        result_df['AVG_DAILY_SALES'] = 0
        result_df['SALES_TRANSACTIONS'] = 0
        result_df['TOTAL_SALES_QTY'] = 0
        result_df['SALES_PERIOD_DAYS'] = 0
    
    # Calculate stock autonomy (days of stock remaining at current sales rate)
    result_df['STOCK_AUTONOMY_DAYS'] = result_df.apply(
        lambda row: (row['CURRENT_STOCK'] / row['AVG_DAILY_SALES']) 
        if row['AVG_DAILY_SALES'] > 0 else float('inf'), axis=1
    )
    # Cap infinity values at 9999 for display purposes
    result_df['STOCK_AUTONOMY_DAYS'] = result_df['STOCK_AUTONOMY_DAYS'].replace(float('inf'), 9999)
    
    # Add site names if available
    if 'sites' in dataframes and sites_df is not None:
        site_names = sites_df[['ID', 'SITE']].drop_duplicates()
        site_names = site_names.rename(columns={'ID': 'SITE', 'SITE': 'SITE_NAME'})
        result_df = result_df.merge(site_names, on='SITE', how='left')
    
    # Add item names and categories if available
    if 'inventory_items' in dataframes and inventory_items_df is not None:
        # Use DESCR1 as primary item name (item description)
        item_info = inventory_items_df[['ITEM', 'DESCR1', 'CATEGORY']].drop_duplicates()
        item_info['ITEM_NAME'] = item_info['DESCR1'].fillna('').astype(str)
        
        # Add category descriptions if available
        if 'categories' in dataframes and categories_df is not None:
            # Get category descriptions from DETDESCR table
            category_descriptions = categories_df[['ID', 'DESCR']].drop_duplicates()
            
            # Convert ID to string to match CATEGORY column type
            category_descriptions['ID'] = category_descriptions['ID'].astype(str)
            category_descriptions = category_descriptions.rename(columns={'ID': 'CATEGORY', 'DESCR': 'CATEGORY_NAME'})
            
            # Ensure CATEGORY column is string type for merge
            item_info['CATEGORY'] = item_info['CATEGORY'].astype(str)
            
            # Merge item info with category descriptions
            item_info = item_info.merge(category_descriptions, on='CATEGORY', how='left')
            item_info['CATEGORY_NAME'] = item_info['CATEGORY_NAME'].fillna('').astype(str)
        else:
            item_info['CATEGORY_NAME'] = ''
        
        # Merge with result_df
        result_df = result_df.merge(
            item_info[['ITEM', 'ITEM_NAME', 'CATEGORY', 'CATEGORY_NAME']], 
            on='ITEM', how='left'
        )
        
        # Fill missing values
        result_df['CATEGORY'] = result_df['CATEGORY'].fillna('')
        result_df['CATEGORY_NAME'] = result_df['CATEGORY_NAME'].fillna('')
    else:
        # If no item data available, create empty columns
        result_df['ITEM_NAME'] = None
        result_df['CATEGORY'] = ''
        result_df['CATEGORY_NAME'] = ''
    
    # Reorder columns to show: Site, Item, Item Description, Category, Current Stock, Total Sales, Avg Sales/Day, Max/Min Daily, Stock Autonomy, Transactions
    final_cols = ['SITE', 'ITEM', 'ITEM_NAME', 'CATEGORY_NAME', 'CURRENT_STOCK', 'TOTAL_SALES_QTY', 'AVG_DAILY_SALES', 'MAX_DAILY_SALES', 'MIN_DAILY_SALES', 'STOCK_AUTONOMY_DAYS', 'SALES_TRANSACTIONS']
    
    # Keep only the columns we want in the final output
    available_cols = [col for col in final_cols if col in result_df.columns]
    result_df = result_df[available_cols]
    
    # Sort by current stock descending
    result_df = result_df.sort_values('CURRENT_STOCK', ascending=False)
    
    if show_details:
        date_range_str = ""
        if from_date or to_date:
            date_range_str = f" (Sales from {from_date or 'start'} to {to_date or 'end'})"
        
        print(f"\n📊 Stock & Sales Analytics{date_range_str}:")
        print(f"Items analyzed: {result_df['ITEM'].nunique():,}")
        print(f"Sites analyzed: {result_df['SITE'].nunique():,}")
        print(f"Total current stock: {result_df['CURRENT_STOCK'].sum():,.0f}")
        print(f"Average stock autonomy: {result_df[result_df['STOCK_AUTONOMY_DAYS'] < 9999]['STOCK_AUTONOMY_DAYS'].mean():.1f} days")
        print(f"Items with positive stock: {(result_df['CURRENT_STOCK'] > 0).sum():,}")
        print(f"Items with sales activity: {(result_df['MAX_DAILY_SALES'] > 0).sum():,}")
        print(f"Items with low stock (< 30 days): {(result_df['STOCK_AUTONOMY_DAYS'] < 30).sum():,}")
        print(f"Total sales transactions: {result_df['SALES_TRANSACTIONS'].sum():,.0f}")
    
    return result_df

def get_stock_and_sales_summary(item_code=None, site_code=None, from_date=None, to_date=None):
    """Quick stock and sales summary with top results"""
    result = calculate_stock_and_sales(item_code, site_code, from_date, to_date, show_details=True)
    if result is not None and not result.empty:
        print(f"\n📋 Top 10 Results:")
        display(result.head(10))
    return result

# Keep backward compatibility
def calculate_stock(item_code=None, site_code=None, show_details=False):
    """Legacy function - use calculate_stock_and_sales for full functionality"""
    return calculate_stock_and_sales(item_code, site_code, show_details=show_details)

print("✅ Enhanced stock and sales calculation functions ready!")
print("\nUsage examples:")
print("• calculate_stock_and_sales('ITEM001', 'SITE001') - Stock and all-time sales for specific item/site")
print("• calculate_stock_and_sales('ITEM001', from_date='2025-01-01', to_date='2025-06-20') - Item sales in date range")
print("• calculate_stock_and_sales(site_code='SITE001', from_date='2025-05-01') - All items at site from May")
print("• calculate_stock_and_sales(from_date='2025-01-01', to_date='2025-03-31') - Q1 sales for all items/sites")
print("• get_stock_and_sales_summary('ITEM001') - Quick overview with top 10 results")
print("\nDate format: 'YYYY-MM-DD' (e.g., '2025-06-20')")

✅ Enhanced stock and sales calculation functions ready!

Usage examples:
• calculate_stock_and_sales('ITEM001', 'SITE001') - Stock and all-time sales for specific item/site
• calculate_stock_and_sales('ITEM001', from_date='2025-01-01', to_date='2025-06-20') - Item sales in date range
• calculate_stock_and_sales(site_code='SITE001', from_date='2025-05-01') - All items at site from May
• calculate_stock_and_sales(from_date='2025-01-01', to_date='2025-03-31') - Q1 sales for all items/sites
• get_stock_and_sales_summary('ITEM001') - Quick overview with top 10 results

Date format: 'YYYY-MM-DD' (e.g., '2025-06-20')


In [ ]:
calculate_stock_and_sales('F001', '14L', '2025-06-01', '2025-06-15')

,SITE,ITEM,ITEM_NAME,CURRENT_STOCK,TOTAL_SALES_QTY,AVG_DAILY_SALES,MAX_DAILY_SALES,MIN_DAILY_SALES,STOCK_AUTONOMY_DAYS,SALES_TRANSACTIONS
0,14L,F001,CORNIERS 25X25X3 MM FAMECO,206.0,17.0,1.133333,5.0,1.0,181.764706,5.0


: 

In [ ]:
# Test connection to troubleshoot webapp issue
print("Testing database connection...")
try:
    test_conn = pyodbc.connect(connection_string)
    print("✅ Connection successful")
    
    # Test a simple query
    cursor = test_conn.cursor()
    cursor.execute("SELECT COUNT(*) FROM ALLSTOCK")
    count = cursor.fetchone()[0]
    print(f"✅ ALLSTOCK table has {count} rows")
    
    test_conn.close()
    print("✅ Connection closed successfully")
except Exception as e:
    print(f"❌ Connection failed: {e}")

In [ ]:
# Check pandas version and groupby approach
import pandas as pd
print(f"Pandas version: {pd.__version__}")

# Test the groupby approach used in the notebook
test_df = sales_details_df.head(100).copy()
test_df['QTY'] = test_df['QTY'].fillna(0)

print("Testing groupby apply approach...")
try:
    # This is the exact approach from the notebook
    result = test_df.groupby('ITEM').apply(lambda x: pd.Series({'count': len(x), 'sum_qty': x['QTY'].sum()}))
    print("✅ Standard groupby.apply() works fine")
    print(f"Result shape: {result.shape}")
except Exception as e:
    print(f"❌ Error with groupby.apply(): {e}")

# Test with warnings
import warnings
warnings.filterwarnings('default')  # Show warnings
result2 = test_df.groupby('ITEM').apply(lambda x: pd.Series({'count': len(x), 'sum_qty': x['QTY'].sum()}))
print("Result with warnings visible completed")

Pandas version: 2.2.3
Testing groupby apply approach...
✅ Standard groupby.apply() works fine
Result shape: (57, 2)
Result with warnings visible completed


C:\Users\User\AppData\Local\Temp\ipykernel_5772\1327024254.py:21: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  result2 = test_df.groupby('ITEM').apply(lambda x: pd.Series({'count': len(x), 'sum_qty': x['QTY'].sum()}))


: 

In [5]:
# Check the structure and data types of category-related columns
print("=== INVENTORY ITEMS (STOCK) TABLE ===")
if inventory_items_df is not None:
    print("Columns:", list(inventory_items_df.columns))
    if 'CATEGORY' in inventory_items_df.columns:
        print(f"CATEGORY column type: {inventory_items_df['CATEGORY'].dtype}")
        print(f"CATEGORY sample values: {inventory_items_df['CATEGORY'].head(10).tolist()}")
        print(f"CATEGORY unique count: {inventory_items_df['CATEGORY'].nunique()}")
    else:
        print("❌ CATEGORY column not found in inventory_items_df")
else:
    print("❌ inventory_items_df is None")

print("\n=== CATEGORIES (DETDESCR) TABLE ===")
if categories_df is not None:
    print("Columns:", list(categories_df.columns))
    if 'ID' in categories_df.columns:
        print(f"ID column type: {categories_df['ID'].dtype}")
        print(f"ID sample values: {categories_df['ID'].head(10).tolist()}")
        print(f"ID unique count: {categories_df['ID'].nunique()}")
    else:
        print("❌ ID column not found in categories_df")
    
    if 'DESCR' in categories_df.columns:
        print(f"DESCR column type: {categories_df['DESCR'].dtype}")
        print(f"DESCR sample values: {categories_df['DESCR'].head(5).tolist()}")
    else:
        print("❌ DESCR column not found in categories_df")
else:
    print("❌ categories_df is None")

=== INVENTORY ITEMS (STOCK) TABLE ===
Columns: ['ITEM', 'DESCR1', 'DESCR2', 'CATEGORY', 'SUBCAT1', 'SUBCAT2', 'WHEREIS', 'SUPPLIER', 'MASDAR', 'PACK', 'BCOSTUS', 'BCOSTLC', 'POSPRICE1', 'SALESCURRID', 'PRICEA', 'PRICEB', 'PRICEC', 'PRICED', 'PRICEE', 'PRICEF', 'BARCODE1', 'BARCODE2', 'BARCODE3', 'BARCODE4', 'PACK1', 'PACK2', 'PACK3', 'PACK4', 'PRICE1', 'PRICE2', 'PRICE3', 'PRICE4', 'MINSTOCK', 'MAXSTOCK', 'GWEIGHT', 'NWEIGHT', 'VOLUME', 'SALESDISC', 'PURCHDISC', 'SUNIT', 'VAT', 'BQTY', 'QTY', 'CDATE', 'YESNO', 'COSTUS', 'COSTLC', 'LCOST', 'STYPE', 'BUY', 'PAY', 'RBUY', 'RPAY', 'ADJUSTIN', 'ADJUSTOUT', 'PRODIN', 'PRODOUT', 'PROFORMAQTY', 'ORDERQTY', 'OQTY', 'JQTY', 'USER_', 'MIZANE', 'MIZANE1', 'MIZANE2', 'MIZANE3', 'MIZANE4', 'OFRE', 'CARTOON', 'LCOST1', 'LCOST2', 'LCOST3', 'LCOST4', 'MYPRN', 'MYCOSTUS', 'MYCOSTLC', 'POSORDER', 'OLDPRICE', 'DISC2', 'SN', 'PROFIT', 'MAXDISC', 'VATRAISON']
CATEGORY column type: object
CATEGORY sample values: ['5074', '5074', '5074', '5074', '5074', '5074

In [6]:
# Check data types for merge compatibility
print("INVENTORY ITEMS CATEGORY column:")
print(f"Type: {inventory_items_df['CATEGORY'].dtype}")
print(f"Sample values: {inventory_items_df['CATEGORY'].dropna().head(3).tolist()}")

print("\nCATEGORIES ID column:")  
print(f"Type: {categories_df['ID'].dtype}")
print(f"Sample values: {categories_df['ID'].dropna().head(3).tolist()}")

# Test merge compatibility
print(f"\nChecking merge compatibility...")
print(f"CATEGORY null count: {inventory_items_df['CATEGORY'].isnull().sum()}")
print(f"ID null count: {categories_df['ID'].isnull().sum()}")

INVENTORY ITEMS CATEGORY column:
Type: object
Sample values: ['5074', '5074', '5074']

CATEGORIES ID column:
Type: int64
Sample values: [1, 2, 3]

Checking merge compatibility...
CATEGORY null count: 5
ID null count: 0


In [8]:
# Test the category merge fix
print("Testing category merge...")
result = calculate_stock_and_sales(from_date='2025-06-01', to_date='2025-06-15')

if result is not None and not result.empty:
    print(f"✅ Function ran successfully! Got {len(result)} rows")
    print(f"Columns: {list(result.columns)}")
    
    # Check if category data is populated
    if 'CATEGORY_NAME' in result.columns:
        non_empty_categories = result[result['CATEGORY_NAME'] != '']
        print(f"Items with category names: {len(non_empty_categories)}")
        if len(non_empty_categories) > 0:
            print("Sample categories:")
            print(non_empty_categories[['ITEM', 'ITEM_NAME', 'CATEGORY_NAME']].head(3))
    
    print(f"\nTop 5 results:")
    print(result.head())
else:
    print("❌ Function returned None or empty result")

Testing category merge...
✅ Function ran successfully! Got 34965 rows
Columns: ['SITE', 'ITEM', 'ITEM_NAME', 'CATEGORY_NAME', 'CURRENT_STOCK', 'TOTAL_SALES_QTY', 'AVG_DAILY_SALES', 'MAX_DAILY_SALES', 'MIN_DAILY_SALES', 'STOCK_AUTONOMY_DAYS', 'SALES_TRANSACTIONS']
Items with category names: 34852
Sample categories:
       ITEM                          ITEM_NAME CATEGORY_NAME
29596  F183           BARRE 10 MM (12M) FAMECO           BAR
2385   F365  TOLE BAC COLOREE 3000X900X0.20 MM        T.GALV
14951  F412               CIMENT 50KG PPC 42.5        CIMENT

Top 5 results:
      SITE    ITEM                            ITEM_NAME CATEGORY_NAME  \
29596  P-O    F183             BARRE 10 MM (12M) FAMECO           BAR   
2385   A-K    F365    TOLE BAC COLOREE 3000X900X0.20 MM        T.GALV   
14951  KIS    F412                 CIMENT 50KG PPC 42.5        CIMENT   
29597  P-O    F184             BARRE 12 MM (12M) FAMECO           BAR   
31760  S-T  KO6700  LAME DE SCIE 18TPI/8D 0.6X300MM CCA    

In [9]:
# Simple test for category merge
try:
    result = calculate_stock_and_sales('F001', '14L', '2025-06-01', '2025-06-15')
    if result is not None and not result.empty:
        print("✅ Category merge fixed! Function runs without error")
        if 'CATEGORY_NAME' in result.columns:
            category_name = result['CATEGORY_NAME'].iloc[0] if len(result) > 0 else 'N/A'
            print(f"Sample category: '{category_name}'")
        print(f"Result columns: {list(result.columns)}")
    else:
        print("⚠️ Function returned empty result")
except Exception as e:
    print(f"❌ Error: {e}")

✅ Category merge fixed! Function runs without error
Sample category: 'CORNIERE'
Result columns: ['SITE', 'ITEM', 'ITEM_NAME', 'CATEGORY_NAME', 'CURRENT_STOCK', 'TOTAL_SALES_QTY', 'AVG_DAILY_SALES', 'MAX_DAILY_SALES', 'MIN_DAILY_SALES', 'STOCK_AUTONOMY_DAYS', 'SALES_TRANSACTIONS']
